In [3]:
import torch
import torch.optim as optim

## torch.optim

torch.optim是实现各种优化算法的包。最常用的方法都已经支持，接口很常规，所以以后也可以很容易地集成更复杂的方法。

### 如何使用optimizer
要使用torch.optim，您必须构造一个optimizer对象。这个对象能保存当前的参数状态并且基于计算梯度更新参数

### 构建
要构造一个Optimizer，你必须给它一个包含参数（必须都是Variable对象）进行优化。然后，您可以指定optimizer的参 数选项，比如学习率，权重衰减等。

例子：

In [ ]:
optimizer1 = optim.SGD(model.parameters(), lr = 0.01, momentum=0.9)
optimizer2 = optim.Adam([var1, var2], lr = 0.0001)

### 为每个参数单独设置选项
Optimizer也支持为每个参数单独设置选项。若想这么做，**不要直接传入Variable的iterable，而是传入dict的iterable。**每一个dict都分别定 义了一组参数，并且包含一个param键，这个键对应参数的列表。如：key value.size()
```
Model's state_dict:
conv1.weight 	 torch.Size([6, 3, 5, 5])
conv1.bias 	 torch.Size([6])
conv2.weight 	 torch.Size([16, 6, 5, 5])
conv2.bias 	 torch.Size([16])
fc1.weight 	 torch.Size([120, 400])
fc1.bias 	 torch.Size([120])
fc2.weight 	 torch.Size([84, 120])
fc2.bias 	 torch.Size([84])
fc3.weight 	 torch.Size([10, 84])
fc3.bias 	 torch.Size([10])
```
其他的键应该optimizer所接受的其他参数的关键字相匹配，并且会被用于对这组参数的 优化。

注意：

您仍然可以将选项作为关键字参数传递。它们将被用作默认值，在不覆盖它们的组中。当您只想改变一个选项，同时保持参数组之间的所有其他选项一致时，这很有用。

例如，当我们想指定每一层的学习率时，这是非常有用的：

In [ ]:
optim.SGD([
            {'params': model.base.parameters()},
            {'params': model.classifier.parameters(), 'lr': 1e-3}
            ], 
            lr=1e-2, momentum=0.9)

这意味着model.base参数将使用默认的学习速率1e-2，model.classifier参数将使用学习速率1e-3，并且0.9的momentum将会被用于所有的参数。

### 进行单次优化
所有的optimizer都会实现step()更新参数的方法。

它能按两种方式来使用：

#### 1.optimizer.step()
这是大多数optimizer所支持的简化版本。一旦梯度被如backward()之类的函数计算好后，我们就可以调用该函数。

例子：

In [ ]:
for input, target in dataset:
    optimizer.zero_grad()
    output = model(input)
    loss = loss_fn(output, target)
    
    loss.backward()
    optimizer.step()

#### 2.optimizer.step(closure)
一些优化算法例如Conjugate Gradient和LBFGS需要重复多次计算函数，因此你需要传入一个闭包去允许它们重新计算你的模型。这个闭包会清空梯度， 计算损失，然后返回。

例子：

In [ ]:
for input, target in dataset:
    def closure():
        optimizer.zero_grad()
        output = model(input)
        loss = loss_fn(output, target)
        loss.backward()
        return loss
    
    optimizer.step(closure)

### 1.SGD算法

### class torch.optim.SGD(params, lr=, momentum=0, dampening=0, weight_decay=0, nesterov=False)

- 实现随机梯度下降算法（momentum可选）。

- Nesterov动量基于On the importance of initialization and momentum in deep learning中的公式.

- 参数：

    - params (iterable) – 用于优化的可以迭代参数或定义参数组
    - lr (float) – 学习率
    - momentum (float, 可选) – 动量因子（默认：0）
    - weight_decay (float, 可选) – 权重衰减（L2范数）（默认：0）
    - dampening (float, 可选) – 动量的抑制因子（默认：0）
    - nesterov (bool, 可选) – 使用Nesterov动量（默认：False）

例子：

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
optimizer.zero_grad()
loss_fn(model(input), target).backward()
optimizer.step()

提示：

带有动量/Nesterov的SGD的实现稍微不同于Sutskever等人以及其他框架中的实现。 考虑到Momentum的具体情况，更新可以写成 v=ρ∗v+g p=p−lr∗v 其中，p、g、v和ρ分别是参数、梯度、速度和动量。 这是在对比Sutskever et. al。和其他框架采用该形式的更新 v=ρ∗v+lr∗g p=p−v Nesterov版本被类似地修改。